<a href="https://colab.research.google.com/github/approjecthub/automation/blob/main/webscrappingExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests

## Getting Category of Games available

In [ ]:
source = requests.get('https://sell.gameloot.in/product-category/sell-games/').text
soup = BeautifulSoup(source, 'lxml')

##getting all category
gameCategory = []
for game in soup.find('ul', class_='woof_list woof_list_checkbox').select('input[type="checkbox"]'):
  gameCategory.append(game['name'])

# PS3, PS VITA, Xbox 360 are not required
notReqCat = ['ps3','ps-vita','xbox-360' ]

reqGameCat = []
for cat in gameCategory:
  if cat not in notReqCat:
    reqGameCat.append(cat)

print(reqGameCat)##Categories that I need

['ps5', 'xbox-series', 'ps4', 'xbox-one', 'nintendo-switch']


## Getting all game datas of above category

In [ ]:
def getDatas(soup, game_ps5, cat):
  product_container = soup.find(id='product_wrapper')
  if not product_container:
    return
  ##get image links 
  for img in product_container.find_all('img'):
    game_ps5['ImgLink'].append(img['src'][2:])
    game_ps5['Category'].append(cat)

  ##get details of product
  for prod in product_container.find_all(class_='product_details'):
    game_ps5['Details'].append(prod.h5.text)

  ##get credit and cash price
  prices = product_container.find_all(class_='woocommerce-Price-amount amount')
  for i in range(len(prices)):
    price = ' '.join(prices[i].text.strip().split())
    
    if (i%2==0):
      game_ps5['CreditPrice'].append(price)
    else:
      game_ps5['CashPrice'].append(price)

In [ ]:


game_ps5 = {}##It will hold all data
game_ps5['ImgLink']= []
game_ps5['Category'] = []
game_ps5['Details'] = []
game_ps5['CreditPrice'] = []
game_ps5['CashPrice'] = []


for cat in reqGameCat:
  source = requests.get(f'https://sell.gameloot.in/product-category/sell-games/?pa_platforms={cat}').text
  soup = BeautifulSoup(source, 'lxml')

  pageCount = 1
  getDatas(soup, game_ps5, cat)

  #Check for more than one pages by observing the pagination
  page_ul = soup.find('ul', class_='page-numbers')
  
  pageCount = 1
  if page_ul:
    pageCount = int(page_ul.find_all('li')[-2].text)

  ##Iterating through all pages
  for pagei in range(1,pageCount):
    source = requests.get(f'https://sell.gameloot.in/product-category/sell-games/page/{pagei+1}/?pa_platforms={cat}').text
    soup = BeautifulSoup(source, 'lxml')
    getDatas(soup, game_ps5, cat)

In [ ]:
##Converting dictionary to dataframe
import pandas as pd
df = pd.DataFrame(game_ps5)

df.head()

,ImgLink,Category,Details,CreditPrice,CashPrice
0,sell.gameloot.in/wp-content/uploads/sites/4/20...,ps5,Ghost of Tsushima Directors Cut PS5,"Rs. 3,900","Rs. 3,650"
1,sell.gameloot.in/wp-content/uploads/sites/4/20...,ps5,Call of Duty Black Ops Cold War PS5,"Rs. 3,750","Rs. 3,500"
2,sell.gameloot.in/wp-content/uploads/sites/4/20...,ps5,NBA 2K22 PS5,"Rs. 3,350","Rs. 3,100"
3,sell.gameloot.in/wp-content/uploads/sites/4/20...,ps5,Call of Duty Black Ops Cold War PS5 (ESR...,"Rs. 3,050","Rs. 2,800"
4,sell.gameloot.in/wp-content/uploads/sites/4/20...,ps5,F1 2021 PS5,"Rs. 3,000","Rs. 2,750"


In [ ]:
df.to_csv('AllGameData.csv', index=False)